This one was quite interesting and challenging for me. Here, I am extracting all the topics and their descriptions from 50 sites listed in by google search. After the data collection phase, the topics are clustered together based on their similarity and exported to a file. This helps to check which topics under the search field are being discussed. While this was primarily made to be used for market analysis, it also works for your trivial queries such as which food items are best for diabetes or what places are best to visit for your holidays.

In [ ]:
#pip install googlesearch-python
from googlesearch import search
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import random

In [ ]:
query = "your search query"

In [ ]:
data = search(query, num_results=50)
#use sleep_interval = 5 if num_results exceeds 100

In [ ]:
data

In [ ]:
web_address = list(data)

In [ ]:
web_address

In [ ]:
web_address[0]

In [ ]:
tech_list = {}
header = ['h1','h2','h3','h4','h5']
for item in web_address:
    print(item)
    count = 0
    try:
        page = requests.get(item, timeout=5)
        soup = BeautifulSoup(page.text, 'lxml')

        for head in header:
            if len(soup.body.find_all(head))<5:
                continue
            else: # soup.find_all(head)!="":
                for key in soup.body.select(head):
                    value = ""
                    
                    for sib in key.find_next_siblings():
                        if sib.name==head:
                            break
                        else:
                            one_para = sib.text
                            value = value + one_para
                            

                            tech_list.update({key.text : value})
                            
                for key in soup.body.select('strong'):
                    one_para = key.find_parent().text
                    value = one_para
                    tech_list.update({key.text : value})

                      
    except:

        print('-----------------')
        print('error:', item)
        print('-----------------')
        pass
print(tech_list.keys())

In [ ]:
df = pd.DataFrame({'Topic':[''], 'Description':['']})

In [ ]:
for item in tech_list.keys():
    print()
    df = df.append({'Topic':item, 'Description':tech_list[item]}, ignore_index = True)

In [ ]:
df.to_excel('filename.xlsx')

In [ ]:
len(tech_list)

In [ ]:
import pandas as pd
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')

In [ ]:
vectorizer = TfidfVectorizer(stop_words = "english")

In [ ]:
tfidf_matrix = vectorizer.fit_transform(tech_list)

In [ ]:
num_clusters = int(len(tech_list)/5)
num_clusters

In [ ]:
kmeans = KMeans(n_clusters=num_clusters, random_state=42)

In [ ]:
kmeans.fit(tfidf_matrix)

In [ ]:
cluster_labels = kmeans.labels_

In [ ]:
clustered_data = {}

In [ ]:
for i, cluster_label in enumerate(cluster_labels):
    if cluster_label not in clustered_data:
        clustered_data[cluster_label]=[]
    clustered_data[cluster_label].append(list(tech_list.keys())[i])

In [ ]:
for cluster_label, news_articles in clustered_data.items():
    print(f"Cluster {cluster_label + 1}:")
    for article in news_articles:
        print(f"- {article}")

In [ ]:
clustered_data

In [ ]:
df = pd.DataFrame({'Topic':[''], 'Description':['']})

In [ ]:
count = 1
for item in clustered_data:
    df = df.append({'Topic':'Cluster'+ str(count), 'Description':'Start'}, ignore_index=True)
    for topic in clustered_data[item]:
        print(str(count) + ': ' + topic)
        print(tech_list.get(topic))
        df = df.append({'Topic':topic, 'Description':tech_list.get(topic)}, ignore_index=True)
    count +=1

In [ ]:
df

In [ ]:
df.to_excel('filename.xlsx')